# Barents Sea example

In this notebook we want to attempt a few things in the Barents Sea as an example:

- TS and volume distributions over 2014
- TS and volume distributinos over the 5-year period
- specific properties and their variation with T and S (T will be on x axis, heat flux sum on y-axis as an example, or summed advective flux on y-axis)
- label water masses or think about what can be seen in these TS diagrams

### Import packages, libraries, functions

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import xarray as xr
import h5py
from scipy.io import loadmat
import matplotlib as mpl
import time
import gsw
from matplotlib.colors import TwoSlopeNorm

# import existing python files
plt.rcParams['figure.figsize'] = (10,4)

# add rdmds reading functions to path
sys.path.append("/home/mmurakami/MITgcm/MITgcm_c68r/MITgcm-checkpoint68r/utils/python/MITgcmutils/MITgcmutils/") # go to parent dir
from mds import *

# add the other files
sys.path.append("/home/mmurakami/crios_backups/an_helper_functions")
from read_binary import *
from calc_UV_conv_1face import calc_UV_conv_1face
from calc_mskmean_T_mod import calc_mskmean_T_mod
from mk3D_mod import mk3D_mod
from aste_helper_funcs import *
from timing_functions import *

### Read grid and masks

In [2]:
# define basin we want
iB = 6   # see mskBasin if you want for later; maybe we should write this to file

In [3]:
dirroot = "/scratch2/atnguyen/aste_270x450x180/"
dirrun = "/scratch/atnguyen/aste_270x450x180/OFFICIAL_ASTE_R1_Sep2019/"
dirIn = dirrun + "diags/BUDG/"
dirDiags = dirrun + "diags/"
dirState = dirDiags + "STATE/"
dirGrid = dirroot + "GRID_real8/"
dirgridnb = dirroot + "GRID_noblank/"
dirgridw = dirroot + "GRID_wet/"

In [4]:
bigaste = True

if bigaste:
    nx = 270
    ncut1 = 450
    ncut2 = 180
else:
    nx = 90
    ncut1 = 150
    ncut2 = 60
    
ny = 2*ncut1+nx+ncut2
nz = 50
nfx = np.array([nx, 0 , nx, ncut2 ,ncut1])
nfy = np.array([ncut1, 0 , nx, nx, nx])

In [5]:
# save myParms from An hard-coding
save_budg_3d = 0
save_budg_2d = 1
save_budg_scalar = 0
save_budg_lev = 0

strbudg = 'Mass'
kBudget = 1
test3d = True
plot_fig = 1
# kz = [[1, 5], [6, 10], [11, 19], [20, 23]]

myparms = {
    'yearFirst': 1979,
    'yearLast': 1979,
    'yearInAv': [1979, 1979],
    'timeStep': 3600,
    'iceModel': 1,
    'useRFWF': 1,
    'useNLFS': 4,
    'rStar': 2,
    'rhoconst': 1029,
    'rcp': 1029 * 3994,
    'rhoi': 910,
    'rhosn': 330,
    'flami': 334000,
    'flamb': 2500000,
    'SIsal0': 4,
    'diagsAreMonthly': 0,
    'diagsAreAnnual': 0,
    'recInAve': [1, 2],
    'SaltPlumeHeatFlux': 0,  # Not sure what this is
    'conserveTr': 0
}

deltaTime = myparms['timeStep']
dt = 86400

In [6]:
# get time-steps:
flist = [f for f in os.listdir(dirIn) if f.startswith('budg2d_snap_set1.') and f.endswith('.data')]
idot = flist[0].index('.')
idot = [idot+1, flist[0][idot+1:].index('.')+idot+1]
idot = np.asarray(idot,dtype=int)

mygrid = {
    'dirGrid': dirGrid,
    'nFaces': 1,
    'fileFormat': 'compact',
    'memoryLimit': 2,
    'ioSize': [nx*ny, 1],
    'facesSize': [ny, nx],
    'facesExpand': [ny, nx],
    'missVal': 0,
}

fldstr2d = ['XC','YC','XG','YG','RAC','Depth','DXG','DYG','DXC','DYC']
fldstr3d = ['hFacC','hFacW','hFacS','mskC','mskS','mskW']
fldstr3dp = ['hFacC','hFacW','hFacS','maskCtrlC','maskCtrlS','maskCtrlW']
fldstr1d = ['RC','RF','DRC','DRF']

for fld in fldstr1d:
    mygrid[fld] = np.squeeze(rdmds(os.path.join(dirGrid, fld)))

for fld in fldstr3d:
    temp = rdmds(os.path.join(dirGrid, fldstr3dp[fldstr3d.index(fld)]))
    mygrid[fld] = temp.reshape(nz, ny, nx)

for fld in fldstr2d:
    temp = rdmds(os.path.join(dirGrid, fld))
    mygrid[fld] = temp.reshape(ny, nx)

mygrid['mskC'][mygrid['mskC'] == 0] = np.nan

areaW, areaS, Vol = [], [], []
for k in range(nz):
    areaW.append(mygrid['DYG'] * mygrid['DRF'][k])
    areaS.append(mygrid['DXG'] * mygrid['DRF'][k])
    Vol.append(mygrid['RAC'] * mygrid['DRF'][k])

In [7]:
# block out obcs
# np tile I think operates the same as repmat in MATLAB
RAC = mygrid['RAC']
RAC3 = np.tile(RAC,(nz,1,1))

hfC = mygrid['hFacC']
DD = mygrid['Depth']
dxg = mygrid['DXG']
dyg = mygrid['DYG']
dxg3d = np.tile(dxg,(nz,1,1))
dyg3d = np.tile(dyg,(nz,1,1))

print(mygrid['DRF'].shape,np.zeros((nz, ny, nx)).shape)
drf3d = mk3D_mod(mygrid['DRF'], np.zeros((nz, ny, nx)))
DD3d = mk3D_mod(DD,np.zeros((nz, ny, nx)))

hfC[hfC == 0] = np.nan
hfC1 = hfC[0, :, :]
hfC1[hfC1 == 0] = np.nan

RACg = RAC * hfC1
hfC1p = np.copy(hfC1)

hfC1p[:, nx-1] = np.nan
hfC1p[ny-1,:] = np.nan
RACgp = RAC * hfC1p

(50,) (50, 1350, 270)


In [8]:
mygrid['hFacC'][mygrid['hFacC'] > 0] = 1
mygrid['hFacC'][mygrid['hFacC'] == 0] = np.nan
hf1 = mygrid['hFacC'][0] # top layer in z

print("hf1",hf1.shape)

hf1 = get_aste_tracer(hf1, nfx, nfy)
# check with hardcoding on this for mini or big aste
if nx == 90:
    hf1[:,281,:] = np.nan
    hf1[:,7,:] = np.nan
    hf1[:,86,122] = np.nan
elif nx == 270:
    hf1[:,844,:] = np.nan
    hf1[:,23,:] = np.nan
    hf1[:,365,260:261] = np.nan

hf1 = aste_tracer2compact(hf1,nfx,nfy)
hf = mygrid["hFacC"]
hf = hf * np.tile(hf1,(nz, 1,1))

hf1 (1350, 270)
shape of tracer fld: (1, 900, 540)
(1, 1350, 270)
compact shape (1, 1350, 270)


In [9]:
# copy basin listing from lookat_layers
fileprefix = "/scratch/pillarh/aste_270x450x180/"
extBasin='run_template/input_maskTransport/'
filename = fileprefix + extBasin + "GATE_transports_v3_osnap.mat"
if nx == 270:
    inf = loadmat(filename)
    mskBasin = (inf["mskBasin"])

mskBasin = mskBasin.T               # python adjustment

# this is now different syntax than the other file
strb=np.array(['CanadaB','ChukchiS','MakarovB','AmundsenB','NansenB','BeringS','BarentsS','GINs','CAA',
               'SPG','LabSea','NPac','NAtlantic','AtlS30'])

mskBasin[mskBasin==50] =6
mskBasin[mskBasin==200]=7
mskBasin[mskBasin==300]=8
mskBasin[mskBasin==400]=9
mskBasin[mskBasin==500]=9
mskBasin[mskBasin==600]=10
mskBasin[mskBasin==700]=11
mskBasin[mskBasin==-1]=12
mskBasin[mskBasin==-100]=13
latNA = 30
lonNA = -82
condition_13 = (mskBasin == 0) & (mygrid['YC'] > latNA) & (mygrid['XC'] > lonNA) & (hf1.reshape((ny,nx)) > 0)
mskBasin[condition_13] = 13
condition_14 = (mskBasin == 0) & (hf1.reshape((ny,nx)) > 0)
mskBasin[condition_14] = 14

mskBasin = mskBasin * hf1
mskBasin = mskBasin[0,:,:]   # change indexing for  python
mskBasin -= 1

# create mskBasin3D to also add to the dataset
mskBasin3D = np.tile(mskBasin[np.newaxis,:,:],(nz,1,1))
mskBasin3D.shape

(50, 1350, 270)

In [10]:
# write the wet mask and also set to nan
wetmask = mygrid['hFacC']
print(wetmask.shape)
wetmask_tracer = get_aste_tracer(wetmask,nfx,nfy)
print("hey",wetmask_tracer.shape)
wetmask_tracer[:,450:,:270] = np.nan
wetmask = aste_tracer2compact(wetmask_tracer,nfx,nfy)
print(wetmask.shape)

(50, 1350, 270)
hey (50, 900, 540)
shape of tracer fld: (50, 900, 540)
(50, 1350, 270)
compact shape (50, 1350, 270)
(50, 1350, 270)


In [11]:
dims = ("z","compact_x","compact_y")
coords = {"z" : np.arange(0,50,1),
          "compact_x" : np.arange(0,1350,1),
          "compact_y" : np.arange(0,270,1)}

attrsBasin = {'title' : 'CanadaB: 0, ChukchiS: 1, MakarovB: 2, AmundsenB: 3, NansenB: 4, BeringS: 5, BarentsS: 6, GINs: 7, CAA: 8, SPG: 9, LabSea: 10, NPac: 11, NAtlantic: 12, AtlS30: 13',
              'standard_name' : 'Basins labeled',
             'units' : 'degree_c'}

attrsWet= {'title' : 'model wet points in compact form',
         'standard_name' : '1= wet, 0=dry',
         'units' : 'boolean'}

# add mskBasin and wetmask to the xarray for the dataset
mskBasin_3D = xr.DataArray(data = mskBasin*wetmask,
                    dims = dims,
                    coords = coords,
                    attrs = attrsBasin)

mskBasin_2D = xr.DataArray(data = mskBasin*wetmask[0,:,:],
                    dims = dims[1:],
                    coords = [coords["compact_x"],coords["compact_y"]],
                    attrs = attrsBasin)

wetmask_xr = xr.DataArray(data = wetmask,
                    dims = dims,
                    coords = coords,
                    attrs = attrsWet)


ds = xr.Dataset()
ds['mskBasin_3D'] = mskBasin_3D
ds['mskBasin_2D'] = mskBasin_2D
ds['wetmask'] = wetmask_xr

In [12]:
# get the number of points in the basin and we can use this as a mask for later
mymsk = mskBasin.copy()
mymsk[mymsk != iB] = np.nan
npoints = len(np.where(mymsk == iB)[0])   # set number of points in the basin to be used later
print(npoints)

10650


### Get the timesteps we want

In [13]:
# get the required time steps

# create an array of the time steps we want to read
# use ts2dte to get december 2014
# first make an array of filenames
dt = 600
startyr = 2002
endyr = 2019

# all the filenames in the system
fnames = get_fnames(dt,startyr,endyr)

times = {}
yearsiwant = np.arange(2009,2011,1)
for year in yearsiwant:
    times[str(year)] = np.arange(1,13,1)
times["2011"] = np.array([1])   # add one year so that we have 5*12 + 1 months for the differencing

# the filenames we want for 2014
tsstr,datetimes = get_tsteps(times,fnames,dt,startyr,1,1)

### Isolate the volumes from ETAN

In [14]:
# ocean and ice
AB_gT=0
debug = False

In [15]:
# ffac = 1e-6
ffac = 1

In [16]:
file_name = 'budg2d_snap_set1'

# assuming all files are structured the same
meta_budg2d_snap_set1 = parsemeta(dirIn + file_name + "." + tsstr[0]+ ".meta")
# try to read this with rec
fldlist = np.array(meta_budg2d_snap_set1['fldList'])
varnames = np.array(['ETAN'])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])

# set for all months
vol = np.zeros((len(tsstr),nz, ny, nx))
print("volume allocation shape",vol.shape)

for t in range(len(tsstr)):
    t1 = tsstr[t]
    #t2 = tsstr[t+1]
    read = [int(t1)]

    # make sure order we write the variables is the same as the order in varnames, else we read the wrong thing
    ETAN,its,meta = rdmds(os.path.join(dirIn, file_name),read,returnmeta=True,rec=recs[0])

    # choose by basin
    # select by existing basin in mskBasin
    ETAN = np.reshape(ETAN,(ny,nx)) * wetmask[0,:,:]

    # ocean
    if debug:
        print(read,its[0],its[1]) # these iteration numbers should be the same as read

    # 3D, with rStar:
    tmpvol = np.zeros((nz, ny, nx))
    if myparms['useNLFS'] < 2 or myparms['rStar'] == 0:        # not this time
        tmpvol[0,:, :] = ETAN * myparms['rhoconst'] * RAC
        if myparms['useRFWF'] == 0:
            tmpvol[0, :, :] = np.zeros((ny, nx))
    else:    # 4/22 look at this one
        if myparms['useRFWF'] != 0:                                 # we are using this  # check if tmp1 is the same as drf3d!!
            tmp1 = mk3D_mod(mygrid['DRF'],hfC) * hfC     # m
            tmp2 = tmp1/mk3D_mod(DD,tmp1)                # drf as a fraction of total depth, this allows us to distribute etan between z
        else:
            tmp2 = drf3d / mk3D_mod(DD, tmp1)

        # this is still wrong, we want to subtract the ETAN anomaly from the existing volumes
        #tmpvol = tmp2 * mk3D_mod(RAC, hfC)                   # existing volume without ETAN
        tmpvol =  (tmp1 + tmp2*mk3D_mod(ETAN, tmp2)) * mk3D_mod(RAC, hfC) * ffac     # volume, m * m^2  = m^3
        #tmpvol += dvol
        #print(np.nanmin(tmpvol))

    vol[t,:,:,:] = tmpvol

print(vol.shape)

volume allocation shape (25, 50, 1350, 270)
(25, 50, 1350, 270)


### Now we can read and bin thetadr and saltdr

In [17]:
dirrun_small = "/scratch2/atnguyen/aste_270x450x180/run_c67w_layers_budget_nlayersorig_advdiffsplit_FULLYfixedXX_checkedwithAN_it0062_nS112_nT112_pk0000631152/"

binsTH = rdmds(dirrun_small + "layers2TH", -1)
binsTH_edges = binsTH[:,:,0].flatten()
binsTH_centers = (binsTH_edges[:-1] + binsTH_edges[1:])/2
nT = binsTH_edges.shape[0]-1

binsSLT = rdmds(dirrun_small + "layers1SLT", -1)
binsSLT_edges = binsSLT[:,:,0].flatten()
binsSLT_centers = (binsSLT_edges[:-1] + binsSLT_edges[1:])/2
nS = binsSLT_edges.shape[0]-1

In [18]:
# read in the initial model T-S fields
# create an array to store the values at all time steps
fileTS = np.zeros((len(tsstr),2,nz,ny,nx))
print(fileTS.shape)

# read each of the files from the list
for t in range(len(tsstr)):
    t1 = tsstr[t]
    read = int(t1)

    # read the file
    thisfileTS = rdmds(dirIn + "budg3d_snap_set2",read)    # dr values, z, y, x
    thisfldListTS = parsemeta(dirIn + "budg3d_snap_set2." + tsstr[t] + ".meta")["fldList"]
    
    thisfileTS = thisfileTS.reshape(2,nz, ny, nx)

    fileTS[t,:,:,:,:] = thisfileTS

(25, 2, 50, 1350, 270)


In [19]:
# we should unweight these before we add them to the dataset
DRF3d = mk3D_mod(mygrid["DRF"],np.zeros((nz, ny, nx)))
temp = fileTS[:,0,:,:,:] / DRF3d
salt = fileTS[:,1,:,:,:] / DRF3d

### Let's mask everything with mskBasin and wetmask just to be certain

In [20]:
vol *= wetmask[np.newaxis,:,:,:]
temp *= wetmask[np.newaxis,:,:,:]
salt *= wetmask[np.newaxis,:,:,:]

In [21]:
# set where not equal to basin to nan
bool_msk = mskBasin!= iB
vol[:,:,bool_msk] = np.nan
temp[:,:,bool_msk] = np.nan
salt[:,:,bool_msk] = np.nan   # shape time, dz, dy, dx

### Now write to dataset

In [22]:
dims = ("iteration","z","compact_x","compact_y")
coords = {"iteration" : datetimes,
          "z" : np.arange(0,50,1),
          "compact_x" : np.arange(0,1350,1),
          "compact_y" : np.arange(0,270,1)}

attrsT = {'title' : 'wighted T data from budg3d_snap_set2 example from official ASTE',
         'standard_name' : 'temperature',
         'units' : 'degree_c'}

attrsS = {'title' : 'weighted S data from budg3d_snap_set2 example from official ASTE',
         'standard_name' : 'salinity',
         'units' : 'ppt'}

attrsVol = {'title' : 'volume calculated from budg2d_snap_set1',
         'standard_name' : 'volume (m^3)',
         'units' : 'ppt'}

In [23]:
temp = xr.DataArray(data = temp,
                    dims = dims,
                    coords = coords,
                    attrs = attrsT)

salt = xr.DataArray(data = salt,
                    dims = dims,
                    coords = coords,
                    attrs = attrsS)

volume = xr.DataArray(data = vol,
                    dims = dims,
                    coords = coords,
                    attrs = attrsVol)

In [24]:
ds = xr.Dataset({'theta': temp, 'salinity': salt, 'volume':volume})
del salt, temp,vol

In [25]:
ds

<xarray.Dataset>
Dimensions:    (iteration: 25, z: 50, compact_x: 1350, compact_y: 270)
Coordinates:
  * iteration  (iteration) datetime64[ns] 2009-01-01 2009-02-01 ... 2011-01-01
  * z          (z) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * compact_x  (compact_x) int64 0 1 2 3 4 5 6 ... 1344 1345 1346 1347 1348 1349
  * compact_y  (compact_y) int64 0 1 2 3 4 5 6 7 ... 263 264 265 266 267 268 269
Data variables:
    theta      (iteration, z, compact_x, compact_y) float64 nan nan ... nan nan
    salinity   (iteration, z, compact_x, compact_y) float64 nan nan ... nan nan
    volume     (iteration, z, compact_x, compact_y) float64 nan nan ... nan nan

### Do the binning in TS - add to array

In [26]:
# let's create a binning function with numpy
def bin_array(arr, bin_edges):
    flattened_arr = arr.flatten()
    bin_indices = np.digitize(flattened_arr, bin_edges)
    bin_indices_3d = bin_indices.reshape(arr.shape)
    
    return bin_indices_3d-1

In [27]:
# is there a way to do this and conserve memory? like maybe we loop through each time step and add it 

In [28]:
# create this as an xarray and add it to the dataset
# now the bins should go up to 112
binned_theta = bin_array(ds.theta.values,binsTH_edges)
binned_theta = binned_theta.astype(float)
binned_theta[binned_theta == nT] = np.nan     # because the binning is setting nan to last value
# print(np.nanmin(binned_theta),np.nanmin(ds.theta.values),binsTH_edges[int(np.nanmin(binned_theta))])
binned_salinity = bin_array(ds.salinity.values,binsSLT_edges)
binned_salinity = binned_salinity.astype(float)
binned_salinity[binned_salinity == nS] = np.nan
# print(np.nanmin(binned_salinity),np.nanmin(ds.salinity.values),binsSLT_edges[int(np.nanmin(binned_salinity))])

binned_temp = xr.DataArray(data = binned_theta*wetmask,
                    dims = dims,
                    coords = coords,
                    attrs = attrsT)

binned_salt = xr.DataArray(data = binned_salinity*wetmask,
                    dims = dims,
                    coords = coords,
                    attrs = attrsS)

In [29]:
ds["theta_binned"] = binned_temp
ds["salinity_binned"] = binned_salt

del binned_temp,binned_salt

In [30]:
del binned_theta, binned_salinity

In [31]:
binwidthT = binsTH_edges[1:] - binsTH_edges[:-1]
binwidthS = binsSLT_edges[1:] - binsSLT_edges[:-1]
dT,dS = np.meshgrid(binwidthT,binwidthS)
dT = dT.reshape(112,112,1)
dS = dS.reshape(112,112,1)

In [32]:
# lets make this into a loop if we can difference in time, we can divide by dT and dS at the end

testmesh = np.zeros((len(ds.iteration.values),nS, nT, npoints))     #ntimes x nS x nT x points in basin
nT = binsTH_edges.shape[0]-1
nS = binsSLT_edges.shape[0]-1

tn = 0
for t in ds.iteration.values:
    print(tn)
    dsx = ds.sel(iteration = t)

    # select the values at this iteration number
    thisvol = dsx.volume.values[:,mskBasin == iB]
    thissalt = dsx.salinity_binned.values[:,mskBasin == iB]
    thistemp = dsx.theta_binned.values[:,mskBasin == iB]

    # trim the fat (nan values)
    thisvol = np.where(np.isnan(thisvol), 0, thisvol)
    thissalt = np.where(np.isnan(thissalt), -1, thissalt)  # Replace NaN with -1
    thistemp = np.where(np.isnan(thistemp), -1, thistemp)  # as above, indexing should not matter because this should be 0 volume

    #print(thisvol.shape,thissalt.shape,thistemp.shape)

    # get m^3/deg C/PSU
    val = thisvol.copy()

    # create the mesh
    meshx = np.zeros((nS, nT, npoints))

    saltflat = thissalt.flatten()
    tempflat = thistemp.flatten()

    # create local timed mesh
    np.add.at(meshx, (thissalt.astype(int), thistemp.astype(int), np.arange(0,npoints,1)), thisvol[...])  # this should work to add at bins
    meshx /= dT   # m^3/deg C
    meshx /= dS   # m^3/deg C/PSU

    # add to big mesh
    testmesh[tn,:,:,:] = meshx

    # delete for memory
    del meshx
    
    tn += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


### Add the testmesh to the dataset, also tendency

In [33]:
testmesh.shape

(25, 112, 112, 10650)

In [34]:
# add the mesh to the dataset
attrsmesh = {'title' : 'mesh for basin with volumes at each T and S bin',
         'standard_name' : 'volume mesh',
         'units' : 'm^3/deg C/PSU'}
dims = ("iteration","i_Tbins","i_Sbins","points")
# not sure if we want the bin centers I think so 
coords = {"iteration" : datetimes,
          "i_Tbins" : np.arange(0,nT,1),
          "i_Sbins" : np.arange(0,nS,1),
          "points" : np.arange(0,npoints,1)}

In [35]:
# syntax
mesh = xr.DataArray(data = testmesh,
                    dims = dims,
                    coords = coords,
                    attrs = attrsmesh)
ds["mesh"] = mesh

In [36]:
del mesh

### Plotting for snap volumes

In [37]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = mpl.cm.get_cmap("viridis").copy()
cmap.set_under(color='white')

/tmp/ipykernel_1445306/1350057631.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap("viridis").copy()


In [38]:
Tbin,Sbin = np.meshgrid(binsTH_edges,binsSLT_edges)

### Find tendency for all of the mesh and take averaging as a set of one year

In [39]:
calc_tend = np.zeros_like(ds.mesh.values[:-1,:,:,0])

In [40]:
for i in range(tsstr.shape[0]-1):
    print(i)
    v2 = np.nansum(ds.mesh.values[i+1,:,:,:],axis=2)
    v1 = np.nansum(ds.mesh.values[i,:,:,:],axis=2)
    timedelta = ds.iteration[i+1].values - ds.iteration[i].values
    dt = timedelta.astype('timedelta64[s]').astype(np.int32)

    thistend = (v2-v1)/dt    # d m^3 / s
    calc_tend[i,:,:] = thistend

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [41]:
# add to dataset
attrsmesh = {'title' : 'mesh for basin with volumes at each T and S bin',
         'standard_name' : 'volume mesh',
         'units' : 'm^3/deg C/PSU'}
dims = ("iteration","i_Tbins","i_Sbins")
# not sure if we want the bin centers I think so 
coords = {"iteration" : datetimes[:-1],
          "i_Tbins" : np.arange(0,nT,1),
          "i_Sbins" : np.arange(0,nS,1)}

In [42]:
# syntax
calc_tend = xr.DataArray(data = calc_tend,
                    dims = dims,
                    coords = coords,
                    attrs = attrsmesh)
ds["calc_tend"] = calc_tend   # because this is of shape datetimes-1, ds['calc_tend'][-1] will be filled with nan

In [43]:
months = set(ds['iteration'].dt.month.values)
months

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [44]:
# now we want to average based on times in iteration - same months

mask = ds['iteration'].dt.month == 1

# Use .sel() with the mask to select only the values where month is 1
ds_month_1 = ds.sel(iteration=mask)

In [45]:
np.nanmean(ds_month_1.calc_tend.values,axis=0).shape

(112, 112)

### Try to plot for all of 2014

In [46]:
path = "/home/mmurakami/crios_backups/ASTE_270/offline_binning/"

In [47]:
years_plt = set(ds['iteration'].dt.year.values)
years_plt = np.array(list(years_plt))
str(years_plt[:2])[1:-1]

'2009 2010'

In [ ]:
num_plots = ds.iteration.values.shape[0]-1  # 12 months
rows = 3
cols = 4
fig,axes = plt.subplots(rows,cols,sharex=True, sharey=True,figsize=(20,16))
if num_plots>1:
    axes = axes.flatten()

for i in range(num_plots-10):
    print(i)
    ax = axes[i]

    # select from dataset
    mask = ds['iteration'].dt.month == i
    ds_month_1 = ds.sel(iteration=mask)

    print("taking axis 0 average of",ds_month_1.calc_tend.values.shape)
    thistend = np.nanmean(ds_month_1.calc_tend.values,axis=0)
    
    # add volume distribution
    title = str(i) + "/" + str(years_plt[:2])[1:-1]
    
    #norm = TwoSlopeNorm(vmin=np.nanmin(thistend), vcenter=0, vmax=np.nanmax(thistend))
    norm = TwoSlopeNorm(vmin=np.nanmin(thistend)/5,vcenter=0,vmax=np.nanmax(thistend)/5)
    cb = ax.pcolormesh(Sbin,Tbin,thistend, norm=norm, cmap=cmap)
    
    # labels
    fig.colorbar(cb,label="m^3/s/deg/PSU")
    ax.set_title(title)
    ax.set_xlabel("S")
    ax.set_ylabel("T")
    ax.grid()
    ax.set_xlim(25,36)
    ax.set_ylim(-2.5,10)

plt.savefig(path + "Barents_tend_2009_2010avg.png",dpi = 300)

0
taking axis 0 average of (0, 112, 112)
1


/tmp/ipykernel_1445306/3044885604.py:17: RuntimeWarning: Mean of empty slice
  thistend = np.nanmean(ds_month_1.calc_tend.values,axis=0)
/tmp/ipykernel_1445306/3044885604.py:23: RuntimeWarning: All-NaN slice encountered
  norm = TwoSlopeNorm(vmin=np.nanmin(thistend)/5,vcenter=0,vmax=np.nanmax(thistend)/5)


taking axis 0 average of (3, 112, 112)
2
